In [1]:
!conda activate asl

'conda' is not recognized as an internal or external command,
operable program or batch file.


# Exam DS807

## Preparation of data

### Labeling Data and converting images to array values

In [3]:
#import packages for labelling and converting imagery data
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import asarray
from sklearn import svm
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier

In [7]:
##############
# Func for creating 1D image arrays

def load_image_function(path):
    images = [] # empty list placeholder
    labels = [] # empty list placeholder
    container = [] # container to validate correct labels
    for filename in os.listdir(path):
        container.append(filename) # add filename to container     
        CCDY_img = load_img(path + f'/{filename}', target_size = (56, 106),
        color_mode="grayscale")                  
        CCDY_img = img_to_array(CCDY_img)
        images.append(CCDY_img) # append the photo to the images. The images list contains a list of arrays       
    return asarray(images), container; # not interested in list of arrays, but array containing lists. Asarray does this. Returns three arrays


In [8]:
################
# Func for creating df with classes and 3d img arrays

def load_img_data ():
    # Start: creatign classes ons string_digits
    # load string digits

    #os.chdir(path_string_digits)

    string_digits = pd.read_csv('DIDA_12000_String_Digit_Labels.csv', 
                 header = None, 
                 names=["index", "string"])
    # create empty class columns
    string_digits['CC'] = 0
    string_digits['D'] = 0
    string_digits['Y'] = 0
    string_digits = string_digits.astype(str)
    # Iterate string digits and append classes
    for i, row in string_digits.iterrows():
        if len(row['string']) != 4:
            row['CC'] = '1'
            row['D'] = '10'
            row['Y'] = '10'
        else:
            row['D'] = row['string'][2]
            row['Y'] = row['string'][3]
            if row['string'][0:2] == '18':
                row['CC']='0'
            else:
                row['CC']='1'
    # End of class labeling on string_digits.
    #
    # Start: create img_df containing scaled images as 1D tensors
    # Convert imagery to 1D arrays with tagged file names

    #os.chdir(path_images)
    image_array, filename = load_image_function('DIDA_12000_String_Digit_Images/DIDA_1')

    # and convert to a img_df
    img_df = pd.DataFrame({'filename': filename, 'img_value': list(image_array)}, 
                          columns=['filename', 'img_value'])
    # Create proper index value in img_df to allow merge on string_digits
    img_df['index'] = img_df['filename']
    for i, row in img_df.iterrows():
        row['index'] = str(img_df['index'][i]).split('.')[0]
    img_df
    # End of creating img_df
    #
    # Start: merge img_df with string_digits
    # match index type on dataframes to merge
    string_digits['index'] = string_digits['index'].astype(int)
    img_df['index'] = img_df['index'].astype(int)
    # Merge dataframes
    df_img_classes = string_digits.merge(img_df)
    # Rearrange order of dataframe
    df_img_classes = df_img_classes.reindex(columns= ['index', 'string', 'CC', 'D', 'Y', 'img_value', 'filename'])
    # End og merging data frames
    return df_img_classes


In [9]:
#load data and shuffle them around
df = load_img_data()
df = df.sample(frac=1).reset_index(drop=True)
df

,index,string,CC,D,Y,img_value,filename
0,6926,1828,0,2,8,"[[[119.0], [123.0], [116.0], [124.0], [123.0],...",6926.jpg
1,926,1838,0,3,8,"[[[108.0], [110.0], [113.0], [112.0], [110.0],...",926.jpg
2,2428,1818,0,1,8,"[[[127.0], [124.0], [126.0], [125.0], [128.0],...",2428.jpg
3,710,1800,0,0,0,"[[[140.0], [138.0], [140.0], [141.0], [137.0],...",710.jpg
4,6470,1839,0,3,9,"[[[137.0], [138.0], [139.0], [134.0], [136.0],...",6470.jpg
...,...,...,...,...,...,...,...
11995,10150,1806,0,0,6,"[[[165.0], [164.0], [163.0], [162.0], [163.0],...",10150.jpg
11996,4300,1825,0,2,5,"[[[218.0], [218.0], [218.0], [220.0], [220.0],...",4300.jpg
11997,8296,1807,0,0,7,"[[[159.0], [160.0], [161.0], [161.0], [162.0],...",8296.jpg
11998,4930,1819,0,1,9,"[[[172.0], [169.0], [169.0], [172.0], [179.0],...",4930.jpg


### Defining fixed train, test and valdiation sets.

In [10]:
df = df.drop(columns=['index','string','filename'])

### Split train, val, test

#### Subsetting df

In [11]:
x_train = df.loc[0:7679]
x_val = df.loc[7680:9599]
x_test = df.loc[9600:11999]

#### Creating labels

In [12]:
#### training labels ###
Y_train = x_train['Y']
Y_train = Y_train.astype(np.uint8)
D_train = x_train['D']
D_train = D_train.astype(np.uint8)
CC_train = x_train['CC']
CC_train = CC_train.astype(np.uint8)

#### validation labels ###
Y_val = x_val['Y']
Y_val = Y_val.astype(np.uint8)
D_val = x_val['D']
D_val = D_val.astype(np.uint8)
CC_val = x_val['CC']
CC_val = CC_val.astype(np.uint8)

#### test labels ###
Y_test = x_test['Y']
Y_test = Y_test.astype(np.uint8)
D_test = x_test['D']
D_test = D_test.astype(np.uint8)
CC_test = x_test['CC']
CC_test = CC_test.astype(np.uint8)


#### Creating image values

In [13]:
# train
values = np.zeros(shape=(len(x_train),56,106,1))
i = 0
j = 0
while i < len(x_train):
    values[i] = x_train['img_value'][j]
    i += 1
    j+=1
x_train = values/255

# val
values = np.zeros(shape=(len(x_val),56,106,1))
i = 0
j = 7680
while i < len(x_val):
    values[i] = x_val['img_value'][j]
    i += 1
    j+=1
x_val = values/255

# test
values = np.zeros(shape=(len(x_test),56,106,1))
i = 0
j = 9600
while i < len(x_test):
    values[i] = x_test['img_value'][j]
    i += 1
    j+=1
x_test = values/255


## Question 1

### 1.2 Non-deeplearning CC-D-Y models

In [14]:
x_train_flat = x_train.reshape((len(x_train), -1))
x_val_flat = x_val.reshape((len(x_val), -1))
x_test_flat = x_test.reshape((len(x_test), -1))

#### SVM, modelCC

In [15]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
Cs = [1,2,3]
Ds = [1,2,3,4]
gammas = ['scale','auto']
decisions = ['ovr', 'ovo']
results = []

for kernel in kernels:
        if kernel == 'poly':
            for C in Cs:
                for D in Ds:
                    for gamma in gammas:
                        for decision in decisions:
                            svm_current = svm.SVC(kernel=kernel, 
                                                    degree=D,
                                                    C =C,
                                                    gamma = gamma,
                                                    random_state = 2022,
                                                    decision_function_shape = decision)
                            svm_current.fit(x_train_flat, CC_train)
                            val_hat = svm_current.predict(x_val_flat)
                            accuracy = accuracy_score(val_hat, CC_val)
                            #
                            results.append([accuracy, kernel, C, gamma, decision, D])
                            print([accuracy, kernel, C, gamma, decision, None])
        else:
            for C in Cs:
                for gamma in gammas:
                    for decision in decisions:
                        svm_current = svm.SVC(kernel=kernel,
                                              C=C,
                                              gamma = gamma,
                                              decision_function_shape = decision,
                                              random_state = 2022)
                        svm_current.fit(x_train_flat, CC_train)
                        val_hat = svm_current.predict(x_val_flat)
                        accuracy = accuracy_score(val_hat, CC_val)
                        #
                        results.append([accuracy, kernel, C, gamma, decision, None])
                        print([accuracy, kernel, C, gamma, decision, None])

results = pd.DataFrame(results)
results.columns = ['Accuracy', 'Kernel','C', 'Gamma', 'Decision', 'Degree']
results

#model = svm.SVC()
#model.fit(x_train_flat, CC_train)
#val_hat = model.predict(x_val_flat)
#accuracy = accuracy_score(val_hat, CC_val)*100
#accuracy

[0.9614583333333333, 'linear', 1, 'scale', 'ovr', None]
[0.9614583333333333, 'linear', 1, 'scale', 'ovo', None]
[0.9614583333333333, 'linear', 1, 'auto', 'ovr', None]
[0.9614583333333333, 'linear', 1, 'auto', 'ovo', None]
[0.9609375, 'linear', 2, 'scale', 'ovr', None]
[0.9609375, 'linear', 2, 'scale', 'ovo', None]
[0.9609375, 'linear', 2, 'auto', 'ovr', None]
[0.9609375, 'linear', 2, 'auto', 'ovo', None]
[0.9609375, 'linear', 3, 'scale', 'ovr', None]
[0.9609375, 'linear', 3, 'scale', 'ovo', None]
[0.9609375, 'linear', 3, 'auto', 'ovr', None]
[0.9609375, 'linear', 3, 'auto', 'ovo', None]
[0.9651041666666667, 'poly', 1, 'scale', 'ovr', None]
[0.9651041666666667, 'poly', 1, 'scale', 'ovo', None]
[0.9625, 'poly', 1, 'auto', 'ovr', None]
[0.9625, 'poly', 1, 'auto', 'ovo', None]
[0.9661458333333334, 'poly', 1, 'scale', 'ovr', None]
[0.9661458333333334, 'poly', 1, 'scale', 'ovo', None]
[0.9625, 'poly', 1, 'auto', 'ovr', None]
[0.9625, 'poly', 1, 'auto', 'ovo', None]
[0.9609375, 'poly', 1, 'sc

,Accuracy,Kernel,C,Gamma,Decision,Degree
0,0.961458,linear,1,scale,ovr,NaN
1,0.961458,linear,1,scale,ovo,NaN
2,0.961458,linear,1,auto,ovr,NaN
3,0.961458,linear,1,auto,ovo,NaN
4,0.960938,linear,2,scale,ovr,NaN
...,...,...,...,...,...,...
79,0.962500,sigmoid,2,auto,ovo,NaN
80,0.962500,sigmoid,3,scale,ovr,NaN
81,0.962500,sigmoid,3,scale,ovo,NaN
82,0.962500,sigmoid,3,auto,ovr,NaN


In [17]:
accuracy_done = accuracy_score(val_hat, CC_val)*100
accuracy_done

96.25

In [19]:
print(str(results[results['Accuracy'] == results['Accuracy'].max()]))

    Accuracy Kernel  C  Gamma Decision  Degree
68  0.972917    rbf  3  scale      ovr     NaN
69  0.972917    rbf  3  scale      ovo     NaN


In [22]:
svm_final = svm.SVC(kernel='rbf',
                      C=3,
                      gamma = 'scale',
                      decision_function_shape = 'ovr',
                      random_state = 2022)
svm_final.fit(np.concatenate((x_train_flat, x_val_flat), axis=0), np.concatenate((CC_train, CC_val), axis=0))
val_hat = svm_final.predict(x_val_flat)
accuracy = accuracy_score(val_hat, CC_val)
accuracy

0.9979166666666667

#### Catboost, modelD

In [16]:
results_catboost_D = []
n_iterations = [100,200,500]
lr = [0.1,0.01]
n_depth = [6,4]
loss_f = 'MultiClass'

for iters in n_iterations:
    for ls in lr:
        for depths in n_depth:
            model_D = CatBoostClassifier(iterations=iters,
                                        learning_rate=ls,
                                        depth=depths,
                                        loss_function=loss_f,
                                        task_type="GPU",
                                        devices='0:1')

            model_D.fit(x_train_flat, Y_train, verbose=50)
            acc = model_D.score(x_val_flat,Y_val)
            # the results from the different hyperparameters are then stored so we can get the best one
            results_catboost_D.append([acc,ls,iters, depths])
results_df_catboost_D = pd.DataFrame(results_catboost_D)
results_df_catboost_D.columns=['accuracy',"learning_rate","n_estimators", "depth"]
#prints the results from best to worst in regards to accuracy, listting the hyperparameters for the result
results_df_catboost_D = results_df_catboost_D.sort_values('accuracy', ascending=False)
results_df_catboost_D

0:	learn: 2.3590970	total: 329ms	remaining: 32.6s
50:	learn: 1.6995672	total: 17.7s	remaining: 17s
99:	learn: 1.4127280	total: 35.8s	remaining: 0us
0:	learn: 2.3687536	total: 152ms	remaining: 15s
50:	learn: 1.9049728	total: 5.75s	remaining: 5.53s
99:	learn: 1.6924843	total: 10.8s	remaining: 0us
0:	learn: 2.3938576	total: 382ms	remaining: 37.8s
50:	learn: 2.2422325	total: 18.7s	remaining: 17.9s
99:	learn: 2.1406637	total: 33.6s	remaining: 0us
0:	learn: 2.3948499	total: 114ms	remaining: 11.3s
50:	learn: 2.2853226	total: 4.77s	remaining: 4.59s
99:	learn: 2.2129804	total: 9.41s	remaining: 0us
0:	learn: 2.3590968	total: 334ms	remaining: 1m 6s
50:	learn: 1.6995669	total: 16.2s	remaining: 47.2s
100:	learn: 1.4093030	total: 31.5s	remaining: 30.8s
150:	learn: 1.2228125	total: 45.8s	remaining: 14.9s
199:	learn: 1.0922946	total: 59.7s	remaining: 0us
0:	learn: 2.3687533	total: 117ms	remaining: 23.3s
50:	learn: 1.9049728	total: 4.74s	remaining: 13.9s
100:	learn: 1.6884748	total: 9.34s	remaining: 9.

,accuracy,learning_rate,n_estimators,depth
0,0.470833,0.10,100,6
1,0.423438,0.10,100,4
2,0.338021,0.01,100,6
3,0.299479,0.01,100,4
4,0.522396,0.10,200,6
5,0.477083,0.10,200,4
6,0.382812,0.01,200,6
7,0.338542,0.01,200,4
8,0.564583,0.10,500,6
9,0.526563,0.10,500,4


In [18]:
results_df_catboost_D.to_csv('Catboost_results_DECADE')

#### Catboost, modelY

In [15]:
results = []
n_iterations = [100,200,500]
lr = [0.1,0.01]
n_depth = [6,4]
loss_f = 'MultiClass'

for iters in n_iterations:
    for ls in lr:
        for depths in n_depth:
            model_Y = CatBoostClassifier(iterations=iters,
                                        learning_rate=ls,
                                        depth=depths,
                                        loss_function=loss_f,
                                        task_type="GPU",
                                        devices='0:1')

            model_Y.fit(x_train_flat, Y_train, verbose=50)
            acc = model_Y.score(x_val_flat,Y_val)
            # the results from the different hyperparameters are then stored so we can get the best one
            results.append([acc,ls,iters, depths])
results_df = pd.DataFrame(results)
results_df.columns=['accuracy',"learning_rate","n_estimators", "depth"]
#prints the results from best to worst in regards to accuracy, listting the hyperparameters for the result
results_df = results_df.sort_values('accuracy', ascending=False)
results_df

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 2.3590973	total: 383ms	remaining: 38s
50:	learn: 1.6995672	total: 18.6s	remaining: 17.8s
99:	learn: 1.4127281	total: 36.3s	remaining: 0us
0:	learn: 2.3687536	total: 135ms	remaining: 13.3s
50:	learn: 1.9049727	total: 5.45s	remaining: 5.24s
99:	learn: 1.6924842	total: 10.7s	remaining: 0us
0:	learn: 2.3938581	total: 386ms	remaining: 38.3s
50:	learn: 2.2422325	total: 19.3s	remaining: 18.6s
99:	learn: 2.1406637	total: 37.3s	remaining: 0us
0:	learn: 2.3948497	total: 121ms	remaining: 12s
50:	learn: 2.2853228	total: 4.55s	remaining: 4.37s
99:	learn: 2.2129804	total: 8.9s	remaining: 0us
0:	learn: 2.3590968	total: 331ms	remaining: 1m 5s
50:	learn: 1.6995670	total: 15.6s	remaining: 45.5s
100:	learn: 1.4093030	total: 31.6s	remaining: 31s
150:	learn: 1.2228128	total: 46.2s	remaining: 15s
199:	learn: 1.0922946	total: 1m 1s	remaining: 0us
0:	learn: 2.3687536	total: 117ms	remaining: 23.3s
50:	learn: 1.9049728	total: 4.78s	remaining: 14s
100:	learn: 1.6884748	total: 9.39s	remaining: 9.21s
150

,accuracy,learning_rate,n_estimators,depth
8,0.564583,0.10,500,6
9,0.526563,0.10,500,4
4,0.522396,0.10,200,6
5,0.477083,0.10,200,4
0,0.470833,0.10,100,6
10,0.440625,0.01,500,6
1,0.423438,0.10,100,4
11,0.409896,0.01,500,4
6,0.382812,0.01,200,6
7,0.338542,0.01,200,4


In [19]:
results_df.to_csv('Catboost_results_YEAR')

## Question 2

In [2]:
#Normalization of gray values
gray_val = gray_val/255
gray_train = gray_train/255
gray_test = gray_test/255
rgb_val = rgb_val/255
rgb_train = rgb_train/255
rgb_test = rgb_test/255

NameError: name 'gray_val' is not defined

### 2.2.A Construct a CNN

### 2.2.B Regularization

### 2.2.C Augmentation

### 2.2.D Transfer learning

## 2.3 Prefered CNN model

## Question 3

### 3.1 Visualization of activation maps

### 3.2 Investigating model performance